In [1]:
import cv2
import os
import pandas as pd
import ast

In [3]:
data_bbox = pd.read_csv('../Data/Detection/bounding_boxes_viavgg_letter.csv')
data_bbox

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,1_original.jpg,398745,{},2,0,"{""name"":""rect"",""x"":441,""y"":231,""width"":32,""hei...",{}
1,1_original.jpg,398745,{},2,1,"{""name"":""rect"",""x"":1217,""y"":786,""width"":31,""he...",{}
2,2_original.jpg,172977,{},1,0,"{""name"":""rect"",""x"":925,""y"":447,""width"":127,""he...",{}
3,3_original.jpg,144855,{},1,0,"{""name"":""rect"",""x"":714,""y"":253,""width"":75,""hei...",{}
4,4_original.jpg,205292,{},1,0,"{""name"":""rect"",""x"":798,""y"":448,""width"":94,""hei...",{}
...,...,...,...,...,...,...,...
600,N286_286_original.jpg,29934,{},1,0,"{""name"":""rect"",""x"":378,""y"":169,""width"":65,""hei...",{}
601,N287_287_original.jpg,32886,{},1,0,"{""name"":""rect"",""x"":388,""y"":228,""width"":56,""hei...",{}
602,N288_288_original.jpg,30154,{},1,0,"{""name"":""rect"",""x"":362,""y"":209,""width"":77,""hei...",{}
603,N289_289_original.jpg,24162,{},1,0,"{""name"":""rect"",""x"":384,""y"":237,""width"":63,""hei...",{}


In [4]:
def xywh_to_xyxy(x_min, y_min, box_width, box_height):
    x_max = x_min + box_width
    y_max = y_min + box_height
    return x_min, y_min, x_max, y_max

def xywh_to_xywh_scaled(x_min, y_min, box_height, box_width, image_width, image_height):
    x_center = x_min + box_width // 2
    y_center = y_min + box_height // 2
    return x_center / image_width, y_center / image_height, box_width / image_width, box_height / image_height

In [11]:
class_num = 1

for i, row in data_bbox.iterrows():
    
    annotation = ast.literal_eval(
        row['region_shape_attributes']
    )

    if not annotation:
        continue

    filename = row['filename'].split('.')[0]
    image = cv2.imread(os.path.join('../Data', 'Detection','yolov4', 'data', 'images', row['filename']))

    width, height = None, None
    try:
        width, height = image.shape[1], image.shape[0]
    except AttributeError:
        print(row)
        pass

    x, y, w, h = xywh_to_xywh_scaled(annotation['x'], annotation['y'], annotation['width'], annotation['height'],
                                             width, height)
    
    
    if os.path.exists(f'../Data/Detection/yolov4/data/images/{filename}.txt'):
        mode = 'a'
        line = f'\n{class_num} {x} {y} {w} {h}'
    else:
        mode = 'w'
        line = f'{class_num} {x} {y} {w} {h}'
        
    with open(f'../Data/Detection/yolov4/data/images/{filename}.txt', mode) as file:
        file.write(line)
        

# Create Train and validation Dataset

In [16]:
import shutil

dataframe = pd.read_csv('../Data/ClassificationCity/city_year_val.csv')
for _, row in dataframe.iterrows():
    image_name = row['ImageName']
    shutil.copy(
        os.path.join('../Data/Detection/yolov4/data/images', image_name),
        os.path.join('../Data/Detection/yolov4/data/val', image_name)
    )

    shutil.copy(
        os.path.join('../Data/Detection/yolov4/data/images', image_name.split('.')[0] + '.txt'),
        os.path.join('../Data/Detection/yolov4/data/val', image_name.split('.')[0] + '.txt')
    )


In [14]:
cv2.imread('../Data/Detection/yolov4/data/images/N28_28_original.jpg')

array([[[110, 105, 105],
        [105, 105, 105],
        [114, 109, 109],
        ...,
        [121, 121, 121],
        [125, 125, 125],
        [121, 121, 121]],

       [[110, 105, 110],
        [110, 105, 110],
        [109, 109, 109],
        ...,
        [121, 121, 121],
        [125, 121, 125],
        [127, 125, 121]],

       [[114, 109, 114],
        [110, 105, 110],
        [105, 105, 105],
        ...,
        [121, 125, 125],
        [121, 121, 121],
        [121, 121, 121]],

       ...,

       [[ 75,  95, 100],
        [ 84,  99, 106],
        [ 75,  95, 100],
        ...,
        [ 75,  95, 100],
        [ 84,  98, 100],
        [ 75,  95, 100]],

       [[ 75,  95, 100],
        [ 84,  99, 106],
        [ 84,  99, 106],
        ...,
        [ 85,  91,  99],
        [ 75,  95, 100],
        [ 75,  95, 100]],

       [[ 84,  99, 106],
        [ 91, 100, 106],
        [ 84,  99, 106],
        ...,
        [ 75,  95, 100],
        [ 75,  95, 100],
        [ 84,  98, 100]]